CONNECT TO WANDB

In [ ]:
!pip install wandb -qU

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB3
import wandb
from wandb.keras import WandbCallback
import shutil

WANDB API LOGIN

In [ ]:
wandb.login()
run_id = ''
wandb.init(id=run_id, resume="must")

In [ ]:
# wandb.init()

In [ ]:
train_x = np.load('data/trainx.npy')
train_y = np.load('data/trainy.npy')
test_x = np.load('data/testx.npy')
test_y = np.load('data/testy.npy')

In [ ]:
NUM_CLASSES = 4
IMG_SIZE = 300
size = (IMG_SIZE, IMG_SIZE)
inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
outputs = EfficientNetB3(include_top=True, weights=None, classes=NUM_CLASSES)(inputs)

checkpoint_path = "model-trained/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [ ]:
model = tf.keras.Model(inputs, outputs)
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.00001, nesterov=True, name="SGD"), loss="categorical_crossentropy", metrics=["accuracy"],jit_compile=True )
model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 300, 300, 3)]     0         
                                                                 
 efficientnetb3 (Functional)  (None, 4)                10789683  
                                                                 
Total params: 10,789,683
Trainable params: 10,702,380
Non-trainable params: 87,303
_________________________________________________________________


IF MODEL IS PRE-TRAINED

In [ ]:
model.load_weights(checkpoint_path)

In [ ]:
with tf.device('/GPU:0'):
  hist = model.fit(train_x, train_y, validation_split=0.2, epochs=5, verbose=2, callbacks=[cp_callback])

In [ ]:
wandb.save('/content/training_1/*ckpt*')
wandb.save(os.path.join(wandb.run.dir, "/content/training_1/checkpoint*"))